# An Introduction to ACEhamiltonians [Archival Version]
This notebook provides a brief guide on how to use the archival [`arXiv.2111.13736`](https://github.com/ACEsuit/ACEhamiltonians.jl/tree/arXiv.2111.13736) snapshot branch of `ACEhamiltonians`. This branch represents the code as it was used at the time of publishing the paper entitled *[Equivariant Analytical Mapping of First Principles Hamiltonians to Accurate and Transferable Materials Models](https://arxiv.org/abs/2111.13736)*. This is intended *only* to allow for external validation of the code as it was used in the paper. Users who wish to make use of the `ACEhamiltonians` package in a more directed capacity should use the [main branch](https://github.com/ACEsuit/ACEhamiltonians.jl/tree/main).

Before proceeding, an `ACEhamiltonians` compatible Julia environment must be setup and correctly configured. This task only needs to be performed once per-user, per-system. To start with, the [`ACEsuit`](https://github.com/ACEsuit) registry must be added to the registry list so that Julia can access and download the packages upon which `ACEhamiltonians` is based. Once this is done, the `ACEhamiltonians` package itself can be downloaded and initialised.

In [ ]:
using Pkg
Pkg.add("IJulia")  # ←┬─ only required when working in a Jupyter notebook
using IJulia       # ←┘
Pkg.Registry.add(RegistrySpec(url="https://github.com/JuliaMolSim/MolSim.git"))  # ← add MolSim registry
Pkg.Registry.add("General")                                                      # ← "General" registry readded to ensure stability
Pkg.add(url="https://github.com/ACEsuit/ACEhamiltonians.jl.git#arXiv.2111.13736")# ← add the ACEhamiltonians package
Pkg.add("JuLIP")

Following this, `ACEhamiltonians` can be imported like so:

In [1]:
using IJulia
using ACEhamiltonians
using ACEhamiltonians.Structure: Data, Params
using ACEhamiltonians.Fitting: params2wmodels
using ACEhamiltonians.Predict: predict_onsite_HS, predict_offsite_HS
using JuLIP: Atoms

## Using ACEhamiltonians
This section provides an introduction to the `ACEhamiltonians` code and how one can build and fit models with it. 

### The Parameters
The `ACE` bases need to be carefully chosen for each specific application. Larger bases can achieve higher accuracy but have a greater risk of losing transferability due to overfitting. Each basis is defined by three parameters, the correlation order _ν_, and the two maximum polynomial degrees _n<sub>max</sub>_ and _l<sub>max</sub>_. The former controls the body-order of the basis while the latter two are used to define degrees of the radial and angular basis functions.

Parameters are specified via a pair of `Params` structures, one for the on-site bases and another for the off-site bases. `ACEhamiltonians` will then used the parameters contained within these structures to construct and fit the model. Parameter must be supplied for each and every basis function; and while such parameters can be freely chosen by the user the number of basis functions is always fixed. This is because the version of `ACEhamiltonians` used in this workbook is hardcoded to model aluminum systems using a _3s2p1d_ basis-set; a restriction that is removed in the release versions.

The required `Params` structures can be instantiated by providing the `Params` constructor with the following six arguments (the exact form of which will be discussed later):
 - `r_cut`: the cutoff distance controls the near/far-sightedness of the on-site `ACE` bases and the interaction cutoff distance of the off-site interactions (Note that for off-site interactions the `ACE` basis environmental cutoff is internally set to one half the specified interaction distance).
 - `mpd`: the maximum polynomial degrees i.e. `mpd` ≡ _n<sub>max</sub>_ ≡ _l<sub>max</sub>_.
 - `ν`: the correlation order controls the body-order of the bases; body-order works out to be `mpd+1` & `mpd+2` for on/off-site bases respectively; i.e. using a global `mpd` value of 1 will result in all on-site interactions being two body and all off-site interactions being three body.
 - `λ`: regularisation values; these values, commonly around 1E-7, control the extent of regularisation used when calculating the coefficients during the fitting process. Larger values result in more aggressive regularisation.
 - `reg_type`: the type of regularisation to use; `1` for ridge regression and `2` for Tikhonov regularization (recommended).
 - `solver`: specifies which solver should be used during fitting; options are `"LSQR"` (recommended), `"QR"`, and `"NaiveSolver"`.

The first four arguments must be vectors of matrices rather than single values; this is because the user must specify one parameter for each and every basis. For on-site parameters, six matrices are required like so: 
```Julia
some_on_site_parameter = Vector{
    Matrix{3,3}, # ss parameters
    Matrix{2,3}, # sp parameters
    Matrix{1,3}, # sd parameters
    Matrix{2,2}, # pp parameters
    Matrix{1,2}, # pd parameters
    Matrix{1,1}  # dd parameters
}
```
The first matrix represents the parameters to be used for the bases representing the interactions between the three sharp bases; the values from top-left to bottom right should be parameters for the s<sub>1</sub>s<sub>1</sub>, s<sub>1</sub>s<sub>2</sub>, s<sub>1</sub>s<sub>3</sub>, s<sub>2</sub>s<sub>3</sub>, s<sub>2</sub>s<sub>2</sub>, ... s<sub>3</sub>s<sub>3</sub> `ACE` bases. The second matrix represents interactions between the three sharp bases and the two primitive bases, and so on and so forth. For the off-site parameters a total of nine matrices are required:
```Julia
some_off_site_parameter = Vector{
    Matrix{3,3}, # ss parameters
    Matrix{2,3}, # sp parameters
    Matrix{1,3}, # sd parameters
    Matrix{3,2}, # ps parameters
    Matrix{2,2}, # pp parameters
    Matrix{1,2}, # pd parameters
    Matrix{3,1}, # ds parameters
    Matrix{2,1}, # dp parameters
    Matrix{1,1}  # dd parameters
}
```
It is **strongly** advised to ensure that the parameter matrices of conjugate interactions are symmetrically equivalent, i.e. the off-site sp parameter matrix is equal to the transpose of the ps parameter matrix. Note that only a single value is to be provided for the last two arguments, `reg_type` and `solver`, as they are applied to all bases. The following code block constructs the `Params` as used in the paper.

In [2]:

# Hamiltonian matrix modelling parameters
# On-site matrices
on_site_H_r_cut = [
    # ss              sp               sd
    fill(10., 3, 3), fill(10., 2, 3), fill(10., 1, 3),
    # pp              pd
    fill(10., 2, 2), fill(10., 1, 2),
    # dd
    fill(10., 1, 1)
]

on_site_H_mpd = [
    # ss            sp             sd
    fill(9, 3, 3), fill(9, 2, 3), fill(9, 1, 3),
    # pp            pd
    fill(9, 2, 2), fill(9, 1, 2),
    # dd
    fill(9, 1, 1)
]

on_site_H_ν = [
    # ss            sp             sd
    fill(2, 3, 3), fill(2, 2, 3), fill(2, 1, 3),
    # pp            pd
    fill(2, 2, 2), fill(2, 1, 2),
    # dd
    fill(2, 1, 1)
]

on_site_H_λ = [
    # ss               sp                sd
    fill(1E-7, 3, 3), fill(1E-7, 2, 3), fill(1E-7, 1, 3),
    # pp               pd
    fill(1E-7, 2, 2), fill(1E-7, 1, 2),
    # dd
    fill(1E-7, 1, 1)
]

# Off-site matrices
off_site_H_r_cut = [
    # ss             sp              sd
    fill(9., 3, 3), fill(9., 2, 3), fill(9., 1, 3),
    # ps             pp              pd
    fill(9., 3, 2), fill(9., 2, 2), fill(9., 1, 2),
    # ds             dp              dd
    fill(9., 3, 1), fill(9., 2, 1), fill(9., 1, 1)
]

off_site_H_mpd = [
    [ # ss
     14 14 14;
     14 14 14;
     14 14  9
    ],
    [ # sp
     14 14 12;
     14 14 10
    ],
    [ # sd
     14 14 11;
    ],
    [ # ps
     14 14;
     14 14;
     12 10
    ],
    [ # pp
     13 13;
     13 13
    ],
    [ # pd
     14 14;
    ],
    [ # sd
     14;
     14;
     11;;
    ],
    [ # dp
     14;
     14;;
    ],
    [ # dd
     14;;
    ]
]

off_site_H_ν = [
    # ss            sp             sd
    fill(1, 3, 3), fill(1, 2, 3), fill(1, 1, 3),
    # ps            pp             pd
    fill(1, 3, 2), fill(1, 2, 2), fill(1, 1, 2),
    # ds            dp             dd
    fill(1, 3, 1), fill(1, 2, 1), fill(1, 1, 1)
]

off_site_H_λ = [
    # ss               sp                sd
    fill(1E-7, 3, 3), fill(1E-7, 2, 3), fill(1E-7, 1, 3),
    # ps               pp                pd
    fill(1E-7, 3, 2), fill(1E-7, 2, 2), fill(1E-7, 1, 2),
    # ds               dp                dd
    fill(1E-7, 3, 1), fill(1E-7, 2, 1), fill(1E-7, 1, 1)
]


# Off-site overlap matrix modelling parameters. On-site overlap elements are taken to be identity matrices and are thus not modelled.
off_site_S_r_cut = [fill(10.0, i, j) for j in 3:-1:1 for i in 3:-1:1]
off_site_S_mpd = [fill(16, i, j) for j in 3:-1:1 for i in 3:-1:1]
off_site_S_ν = [fill(0, i, j) for j in 3:-1:1 for i in 3:-1:1]
off_site_S_λ = [fill(1E-7, i, j) for j in 3:-1:1 for i in 3:-1:1]


# Params structure instantiation
on_site_H_parameters = Params(on_site_H_r_cut, on_site_H_mpd, on_site_H_ν, on_site_H_λ, 2, "LSQR")
off_site_H_parameters = Params(off_site_H_r_cut, off_site_H_mpd, off_site_H_ν, off_site_H_λ, 2, "LSQR")
off_site_S_parameters = Params(off_site_S_r_cut, off_site_S_mpd, off_site_S_ν, off_site_S_λ, 2, "LSQR");

### The Data
Now that the parameters of the model have been defined, the data on which it is to be fitted must be chosen. This is done through the use of the `Data` structure which. The `Data` structure takes two arguments `file_names` and `atom_blocks`. The former argument provides a list of HDF5 files from which to extract data. Each file should contain the results of exactly one DFT reference calculation. The latter argument specifies from which atom-blocks fitting data should be taken, this is done to avoid fitting on redundant data as is commonly found in Hamiltonian matrices produced from supercell calculation.

For off-site `Data` structures a block is specified by a tuple of atomic indices like so `(i, j)`, where `(1, 2)` would indicate the atom-block associated with the interaction between atoms 1 and 2. For on-site `Data` structures only a single index is required per-block as they always lie on the diagonal, thus `i` will resolve to `(i, i)` internally. It is worth pointing out that the `Data` object does not actually load any data only tells lower levels of the code which files and atom-blocks are to be used during fitting. The actual loading of the data and Hamiltonian vs overlap matrix selection is done by the code as and when needed.

If `atom_blocks` is a vector of vectors, then it is assumed that the i'th vector in `atom_blocks` specifies the atom-blocks to be extracted from the i'th file in `file_names`. However, if `atom_blocks` is a single vector then it is assumed that those atom-blocks should be extracted from all files. The following code block demonstrates a pair of `Data` structures may be instantiated:

In [3]:
# Load all on-site blocks from the three example Al3 systems
on_site_data = Data(["Data/Al3_1.h5", "Data/Al3_2.h5", "Data/Al3_3.h5"], [1, 2, 3])
# Load all symmetrically inequivalent off-site atom-blocks
off_site_data = Data(["Data/Al3_1.h5", "Data/Al3_2.h5", "Data/Al3_3.h5"], [(1, 2), (1, 3), (2, 3)]);

### The Model
With the model parameters defined and the fitting data selected the `ACEhamiltonians` model can be constructed. It should be noted that while this code version _i_) combines the model construction and fitting operations and _ii_) requires on/off-site models to be handled separately; such issues are resolved in the developmental versions. Model construction and fitting is carried out via the `params2wmodels` function. This function takes the relevant `Data` and `Params` structures and returns a pair of models along with the data on which they were fitted, like so:

In [4]:
# construct and fit the off-site Hamiltonian model.
H_off_site_model, S_off_site_model, off_site_fitting_data = params2wmodels(off_site_data, off_site_H_parameters, off_site_S_parameters)

# Construct and fit the on-site Hamiltonian model. Observe how a fake overlap parameter set must be provided during fitting.
H_on_site_model, _, on_site_fitting_data = params2wmodels(on_site_data, on_site_H_parameters, on_site_H_parameters);

### Using the Models

#### Custom Models
Once the `ACEhamiltonians` models have been constructed and fitted they may be evaluated to predict new atom-blocks. Predictions can be made by providing either the `predict_onsite_HS` or `predict_offsite_HS` function with a model, with which predictions are to be made, a `JuLIP.Atoms` instance, on which predictions are to be made, and the blocks for which for which predictions are to be calculated, like so:

In [26]:
atoms = Atoms(;Z=[13, 13, 13], X=[[-1.,0, 0],[1.,0,0], [5.,1.,0]], cell=[10. 0 0; 0 10 0; 0 0 10])
H_on_site_blocks = predict_onsite_HS(atoms, H_on_site_model, [1, 2, 3])
H_off_site_blocks = predict_offsite_HS(atoms, H_off_site_model, [(1, 2), (1, 3), (2, 3)])
S_off_site_blocks = predict_offsite_HS(atoms, S_off_site_model, [(1, 2), (1, 3), (2, 3)]);

Predicting block 1 2
Predicting block 1 3
Predicting block 2 3
Predicting block 1 2
Predicting block 1 3
Predicting block 2 3


The target blocks are specified in the same way as for the `Data` structures. It is **critical** to note that the results returned by the `predict_offsite_HS` are only valid when symmetrised with its symmetric counterpart. That is to say, to construct the atom block (1,2) the following code is required: 

In [27]:
block_12 = (predict_offsite_HS(atoms, H_off_site_model, [(2, 1)])[:, :, 1] + predict_offsite_HS(atoms, H_off_site_model, [(1, 2)])[:, :, 1]') / 2

Predicting block 2 1
Predicting block 1 2


14×14 Matrix{Float64}:
  4.38829e-14   5.75177e-5   -0.186716     …   0.0          -0.539439
  5.67031e-5   -0.000802951   0.130391         0.0           0.323229
 -0.186711      0.13042      -0.135406         0.0          -0.0647238
 -5.23901e-6    9.68193e-6   -0.000137379      0.0           0.000159587
  0.0           0.0           0.0              0.0760883     0.0
 -0.000634079   0.00183569   -0.0566533    …   0.0          -0.062203
  0.000477264  -0.000284889  -0.000616997      0.0          -0.000282373
  0.0           0.0           0.0             -0.0450913     0.0
  0.431503     -0.278592      0.129761         0.0          -0.0151455
  0.000595332  -0.000416186   0.000380376      0.0           0.000641407
  0.0           0.0           0.0          …  -2.28375e-5    0.0
  0.313126     -0.187618      0.0370826        0.0          -0.0139063
  0.0           0.0           0.0              0.000195808   0.0
 -0.539439      0.323229     -0.0647238        0.0           0.00176197

#### Precompiled Models
Once instantiated, models can be saved to JSON formatted files for later recall. This saves having to construct and fit a model each time that it is required. These models can be loaded via the `recover_model` function like so:

In [1]:
using ACE2tb.Dictionary: recover_model

H_on_site_model = recover_model("H_on_site_model_file_name.json")
H_off_site_model = recover_model("H_off_site_model_file_name.json")
S_off_site_model = recover_model("S_off_site_model_file_name.json")

The models used to produce the results discussed in the [associated paper](https://arxiv.org/abs/2111.13736) can be found online [here](https://doi.org/10.5281/zenodo.6561452).